In [75]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("C:\\Users\\lenovo\\Downloads\\balanced_processed_covid19_tweets_copy.csv")

# Select the 10th and 14th columns
selected_columns = df.iloc[:, [0, 1]]

# Create a new DataFrame with the selected columns
df = pd.DataFrame(selected_columns)

# Display the new DataFrame
df.head()  # You can adjust the number of rows to display if needed


,text,sentiment
0,covid change work general recruiting specifica...,Neutral
1,wear face covering shopping includes visit loc...,Neutral
2,order logo graphicdesigner logodesign logodesi...,Neutral
3,rajasthan government today started plasma bank...,Neutral
4,nagaland police covid awareness city tower jun...,Neutral


In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer


corpus = df['text'].tolist()

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=100)  # You can adjust max_features as needed

# Fit and transform the corpus
tfidf_features = tfidf_vectorizer.fit_transform(corpus)

# Convert TF-IDF features to an array
tfidf_features = tfidf_features.toarray()


In [50]:
type(tfidf_features)

numpy.ndarray

In [51]:
import numpy as np
import pandas as pd



# Load GloVe word vectors
def load_glove_embeddings(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        word_to_vec = {}
        for line in f:
            values = line.split()
            word = values[0]
            vec = np.array(values[1:], dtype='float32')
            word_to_vec[word] = vec
    return word_to_vec

glove_embeddings = load_glove_embeddings("C:\\Users\\lenovo\\My_Folders\\7th_sem\\IT350\\IT350_Project\\glove.6B.100d.txt")

# Convert tweets to GloVe embeddings
def get_average_embedding(tweet, word_embeddings):
    words = tweet.split()
    embeddings = [word_embeddings.get(word, np.zeros(100)) for word in words]
    return np.mean(embeddings, axis=0)

# Apply the embedding function to the 'text' column
glove_features = df['text'].apply(lambda x: get_average_embedding(x, glove_embeddings))

# Now, 'glove_features' contains the GloVe embeddings for each tweet
# It's a Series of NumPy arrays

# Convert 'glove_features' to a NumPy array
glove_features = np.vstack(glove_features)



In [52]:
glove_features

array([[-1.32310221e-01, -2.96978884e-02, -9.70799939e-02, ...,
        -9.88055550e-02,  3.64647778e-01,  1.87468888e-01],
       [-1.75629362e-01,  1.37968436e-01, -5.63077740e-02, ...,
        -1.89345628e-01,  8.43718410e-01,  2.04985008e-01],
       [-7.86462612e-03,  2.63751378e-01, -2.70341262e-02, ...,
        -4.34415875e-01,  1.93508500e-01,  6.83593136e-02],
       ...,
       [ 2.93759964e-02,  1.26605728e-01,  3.18121452e-01, ...,
         2.20753635e-01,  2.52883728e-01,  2.99689997e-01],
       [-2.73862481e-02,  1.97214626e-01,  3.04135006e-01, ...,
         8.03251285e-04,  5.85537124e-03,  3.84788755e-01],
       [ 1.22584984e-01,  1.47826836e-01,  4.29515660e-01, ...,
         1.21236399e-01,  2.97618002e-01,  7.27383047e-02]])

In [53]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize




# Tokenize tweets
tokenized_tweets = [word_tokenize(tweet.lower()) for tweet in df['text']]

# Train Word2Vec model
word2vec_model = Word2Vec(tokenized_tweets, vector_size=100, window=5, min_count=1, sg=0)

# Function to get the word embedding for a tweet
def get_average_word2vec_embedding(tweet, word2vec_model):
    words = word_tokenize(tweet.lower())
    embeddings = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    return np.mean(embeddings, axis=0) if embeddings else np.zeros(100)

# Apply the embedding function to the 'text' column
word2vec_features = df['text'].apply(lambda x: get_average_word2vec_embedding(x, word2vec_model))

# Convert 'word2vec_features' to a NumPy array
word2vec_features = np.vstack(word2vec_features)

In [54]:
word2vec_features

array([[-0.01437218,  0.01492738,  0.00561952, ..., -0.01801079,
         0.00760741,  0.00946161],
       [-0.00631607,  0.01063508,  0.00762183, ..., -0.00848229,
         0.00297726,  0.00186848],
       [ 0.00183554,  0.00161727,  0.00221491, ..., -0.00221081,
         0.00103854, -0.002015  ],
       ...,
       [-0.00827947,  0.00967096,  0.00502178, ..., -0.01215699,
         0.00604552,  0.00641924],
       [-0.01602875,  0.01705617,  0.00861055, ..., -0.02190098,
         0.00384176,  0.0075661 ],
       [-0.00801119,  0.00750385,  0.00094066, ..., -0.01110046,
        -0.00021131,  0.00454398]], dtype=float32)

In [55]:
from gensim.models import FastText
from gensim.utils import simple_preprocess
import pandas as pd



# Train a FastText model
model = FastText(sentences=df['text'].apply(lambda x: simple_preprocess(x)), vector_size=100, window=5, min_count=1, sg=1)

# Function to get the FastText embedding for a sentence
def get_fasttext_embedding(sentence):
    words = simple_preprocess(sentence)
    return sum([model.wv[word] for word in words if word in model.wv])

# Apply the function to your DataFrame and get the embeddings separately
fasttext_embeddings = df['text'].apply(get_fasttext_embedding)

# Now, fasttext_embeddings is a Series containing FastText-like embeddings for each tweet.
# Each element of the series is a numpy array representing the embedding.
fasttext_embeddings = np.vstack(fasttext_embeddings)

In [56]:
fasttext_embeddings

array([[ 0.09200902,  1.7944399 , -4.890356  , ..., -0.47877878,
         1.0281978 ,  3.1277945 ],
       [ 0.09266106,  1.7999165 , -4.9416203 , ..., -0.4780107 ,
         1.0316372 ,  3.1708076 ],
       [ 0.05594861,  1.1404275 , -3.1194494 , ..., -0.31172308,
         0.6630717 ,  1.999525  ],
       ...,
       [ 0.09603129,  1.6566579 , -4.5401177 , ..., -0.45983636,
         0.9653411 ,  2.912333  ],
       [ 0.07413157,  1.3950788 , -3.7668443 , ..., -0.38317403,
         0.78956354,  2.412159  ],
       [ 0.10726417,  2.0525918 , -5.5727043 , ..., -0.54822123,
         1.1806769 ,  3.5759623 ]], dtype=float32)

In [76]:
df['tfidf_features']=tfidf_features.tolist()
df['glove_features']=glove_features.tolist()
df['word2vec_features']=word2vec_features.tolist()
df['fasttext_embeddings']=fasttext_embeddings.tolist()
df.head()

,text,sentiment,tfidf_features,glove_features,word2vec_features,fasttext_embeddings
0,covid change work general recruiting specifica...,Neutral,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.13231022138562468, -0.02969788842731052, -...","[-0.01437218114733696, 0.014927376993000507, 0...","[0.09200902283191681, 1.7944399118423462, -4.8..."
1,wear face covering shopping includes visit loc...,Neutral,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.1756293624639511, 0.13796843588352203, -0....","[-0.006316069047898054, 0.010635077022016048, ...","[0.0926610603928566, 1.7999165058135986, -4.94..."
2,order logo graphicdesigner logodesign logodesi...,Neutral,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.007864626124501228, 0.26375137781724334, -...","[0.0018355390056967735, 0.0016172656323760748,...","[0.055948611348867416, 1.1404274702072144, -3...."
3,rajasthan government today started plasma bank...,Neutral,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.15869990789464541, -0.146028643740075, 0.1...","[-0.012957141734659672, 0.01843681000173092, 0...","[0.18618302047252655, 2.992691993713379, -8.20..."
4,nagaland police covid awareness city tower jun...,Neutral,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.03638550266623497, -0.22403829880058765, 0....","[-0.015640147030353546, 0.023020263761281967, ...","[0.08334425091743469, 1.768351674079895, -4.75..."


In [77]:
# Define labels mapping
label_mapping = {'Joy': 0, 'Neutral': 1, 'Fear': 2, 'Sad': 3, 'Anger': 4}

# Convert sentiment labels to numerical labels
df['sentiment'] = df['sentiment'].map(label_mapping)


In [59]:
df.head()

,text,sentiment,tfidf_features,glove_features,word2vec_features,fasttext_embeddings
0,covid change work general recruiting specifica...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.13231022138562468, -0.02969788842731052, -...","[-0.01437218114733696, 0.014927376993000507, 0...","[0.09200902283191681, 1.7944399118423462, -4.8..."
1,wear face covering shopping includes visit loc...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.1756293624639511, 0.13796843588352203, -0....","[-0.006316069047898054, 0.010635077022016048, ...","[0.0926610603928566, 1.7999165058135986, -4.94..."
2,order logo graphicdesigner logodesign logodesi...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.007864626124501228, 0.26375137781724334, -...","[0.0018355390056967735, 0.0016172656323760748,...","[0.055948611348867416, 1.1404274702072144, -3...."
3,rajasthan government today started plasma bank...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.15869990789464541, -0.146028643740075, 0.1...","[-0.012957141734659672, 0.01843681000173092, 0...","[0.18618302047252655, 2.992691993713379, -8.20..."
4,nagaland police covid awareness city tower jun...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.03638550266623497, -0.22403829880058765, 0....","[-0.015640147030353546, 0.023020263761281967, ...","[0.08334425091743469, 1.768351674079895, -4.75..."


In [60]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [33]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Reshape
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, matthews_corrcoef, f1_score

# Assuming you have a DataFrame named df

# Extract features and labels
X_tfidf = np.array(df['tfidf_features'].tolist())

y = df['sentiment']

# Reshape the data to have a third dimension
X_tfidf = X_tfidf.reshape(X_tfidf.shape[0], X_tfidf.shape[1], 1)


# Split data into train and test sets
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)
# Repeat the above steps for other feature types

# # Define GRU model
# model_tfidf = Sequential([
#     GRU(128, input_shape=(X_tfidf.shape[1], X_tfidf.shape[2]), activation='relu'),
#     Dense(5, activation='softmax')
# ])

#Dense layer added to GRU layer
model_tfidf = Sequential([
    Reshape((X_tfidf.shape[1], 1), input_shape=(X_tfidf.shape[1],)),  # Reshape for GRU input
    GRU(128, activation='relu'),
    Dropout(0.5),  # Adding dropout for regularization
    Dense(64, activation='relu'),
    Dropout(0.3),  # Adding dropout for regularization
    Dense(5, activation='softmax')
])
# Compile the model
model_tfidf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_tfidf.fit(X_train_tfidf, y_train, epochs=10, validation_data=(X_test_tfidf, y_test))

# Evaluate the model
y_pred_tfidf = np.argmax(model_tfidf.predict(X_test_tfidf), axis=-1)
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)
recall_tfidf = recall_score(y_test, y_pred_tfidf, average='weighted')
mcc_tfidf = matthews_corrcoef(y_test, y_pred_tfidf)
f1_tfidf = f1_score(y_test, y_pred_tfidf, average='weighted')

print(y_pred_tfidf)
print(accuracy_tfidf)
print(mcc_tfidf)
print(f1_tfidf)

Epoch 1/10
52/52 [==============================] - 10s 133ms/step - loss: 1.6093 - accuracy: 0.1939 - val_loss: 1.6104 - val_accuracy: 0.2009
Epoch 2/10
52/52 [==============================] - 7s 133ms/step - loss: 1.6096 - accuracy: 0.2031 - val_loss: 1.6103 - val_accuracy: 0.2009
Epoch 3/10
52/52 [==============================] - 7s 130ms/step - loss: 1.6101 - accuracy: 0.2000 - val_loss: 1.6098 - val_accuracy: 0.2009
Epoch 4/10
36/52 [===================>..........] - ETA: 1s - loss: 1.6093 - accuracy: 0.2031

KeyboardInterrupt: 

In [27]:
joy_train=0
neutral_train=0
fear_train=0
sad_train=0
anger_train=0
for i in y_train:
    if(i==0):
        joy_train+=1
    elif(i==1):
        neutral_train+=1
    elif(i==2):
        fear_train+=1
    elif(i==3):
        sad_train+=1
    elif(i==4):
        anger_train+=1

print('Joy_train : ',joy_train)
print('Neutral_train : ',neutral_train)
print('fear_train : ',fear_train)
print('sad_train : ',sad_train)
print('anger_train : ',anger_train)
        

Joy_train :  333
Neutral_train :  310
fear_train :  338
sad_train :  335
anger_train :  319


In [26]:
y_train

1316    2
1934    4
1914    4
1602    3
436     1
       ..
1638    3
1095    2
1130    2
1294    2
860     0
Name: sentiment, Length: 1635, dtype: int64

In [23]:
y_train[][1]

SyntaxError: invalid syntax (3078182160.py, line 1)

In [17]:
print(y_test)

1720    3
2025    4
1430    3
2320    4
2141    4
       ..
1905    4
294     1
1822    3
588     0
1887    3
Name: sentiment, Length: 702, dtype: int64


In [35]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, matthews_corrcoef, f1_score

# Assuming you have a DataFrame named df

# Extract features and labels
X_glove = np.array(df['glove_features'].tolist())
y = df['sentiment']

# Reshape the data to have a third dimension
X_glove = X_glove.reshape(X_glove.shape[0], X_glove.shape[1], 1)

# Split data into train and test sets
X_train_glove, X_test_glove, y_train, y_test = train_test_split(X_glove, y, test_size=0.2, random_state=42)

# Define GRU model
model_glove = Sequential([
    GRU(128, input_shape=(X_glove.shape[1], X_glove.shape[2]), activation='relu'),
    Dense(5, activation='softmax')
])

# Compile the model
model_glove.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_glove.fit(X_train_glove, y_train, epochs=10, validation_data=(X_test_glove, y_test))

# Evaluate the model
y_pred_glove = np.argmax(model_glove.predict(X_test_glove), axis=-1)
accuracy_glove = accuracy_score(y_test, y_pred_glove)
recall_glove = recall_score(y_test, y_pred_glove, average='weighted')
mcc_glove = matthews_corrcoef(y_test, y_pred_glove)
f1_glove = f1_score(y_test, y_pred_glove, average='weighted')

Epoch 1/10
59/59 [==============================] - 10s 126ms/step - loss: 1.6085 - accuracy: 0.2076 - val_loss: 1.6034 - val_accuracy: 0.2457
Epoch 2/10
59/59 [==============================] - 7s 121ms/step - loss: 1.6041 - accuracy: 0.2354 - val_loss: 1.5975 - val_accuracy: 0.2692
Epoch 3/10
59/59 [==============================] - 7s 127ms/step - loss: 1.5978 - accuracy: 0.2472 - val_loss: 1.5867 - val_accuracy: 0.2650
Epoch 4/10
59/59 [==============================] - 7s 123ms/step - loss: 1.5914 - accuracy: 0.2541 - val_loss: 1.5761 - val_accuracy: 0.2735
Epoch 5/10
59/59 [==============================] - 7s 124ms/step - loss: 1.5807 - accuracy: 0.2595 - val_loss: 1.5697 - val_accuracy: 0.2543
Epoch 6/10
59/59 [==============================] - 7s 123ms/step - loss: 1.5779 - accuracy: 0.2713 - val_loss: 1.5564 - val_accuracy: 0.2735
Epoch 7/10
59/59 [==============================] - 7s 126ms/step - loss: 1.5717 - accuracy: 0.2745 - val_loss: 1.5594 - val_accuracy: 0.2821
Epoch

In [36]:
model_glove.fit(X_train_glove, y_train, epochs=20, validation_data=(X_test_glove, y_test))

Epoch 1/20
59/59 [==============================] - 7s 120ms/step - loss: 1.5359 - accuracy: 0.3007 - val_loss: 1.5333 - val_accuracy: 0.2927
Epoch 2/20
59/59 [==============================] - 7s 116ms/step - loss: 1.5160 - accuracy: 0.3264 - val_loss: 1.5120 - val_accuracy: 0.3248
Epoch 3/20
59/59 [==============================] - 7s 119ms/step - loss: 1.5017 - accuracy: 0.3242 - val_loss: 1.5216 - val_accuracy: 0.2927
Epoch 4/20
59/59 [==============================] - 7s 117ms/step - loss: 1.4780 - accuracy: 0.3462 - val_loss: 1.5010 - val_accuracy: 0.3056
Epoch 5/20
59/59 [==============================] - 7s 118ms/step - loss: 1.4737 - accuracy: 0.3515 - val_loss: 1.5550 - val_accuracy: 0.2863
Epoch 6/20
59/59 [==============================] - 7s 117ms/step - loss: 1.4559 - accuracy: 0.3649 - val_loss: 1.5088 - val_accuracy: 0.3034
Epoch 7/20
59/59 [==============================] - 7s 118ms/step - loss: 1.4511 - accuracy: 0.3547 - val_loss: 1.5162 - val_accuracy: 0.3120
Epoch 

In [37]:
model_glove.fit(X_train_glove, y_train, epochs=100, validation_data=(X_test_glove, y_test))

Epoch 1/100
59/59 [==============================] - 7s 119ms/step - loss: 1.3871 - accuracy: 0.4013 - val_loss: 1.4968 - val_accuracy: 0.3526
Epoch 2/100
59/59 [==============================] - 7s 120ms/step - loss: 1.3875 - accuracy: 0.4045 - val_loss: 1.4866 - val_accuracy: 0.3397
Epoch 3/100
59/59 [==============================] - 7s 118ms/step - loss: 1.3807 - accuracy: 0.4061 - val_loss: 1.5292 - val_accuracy: 0.3697
Epoch 4/100
59/59 [==============================] - 7s 117ms/step - loss: 1.3745 - accuracy: 0.4173 - val_loss: 1.5201 - val_accuracy: 0.3526
Epoch 5/100
59/59 [==============================] - 7s 118ms/step - loss: 1.3700 - accuracy: 0.4157 - val_loss: 1.5085 - val_accuracy: 0.3697
Epoch 6/100
59/59 [==============================] - 7s 116ms/step - loss: 1.3686 - accuracy: 0.4104 - val_loss: 1.4925 - val_accuracy: 0.3697
Epoch 7/100
59/59 [==============================] - 7s 118ms/step - loss: 1.3677 - accuracy: 0.4125 - val_loss: 1.5070 - val_accuracy: 0.3397

Epoch 58/100
59/59 [==============================] - 7s 119ms/step - loss: 1.1631 - accuracy: 0.5152 - val_loss: 1.6143 - val_accuracy: 0.3504
Epoch 59/100
59/59 [==============================] - 7s 119ms/step - loss: 1.1437 - accuracy: 0.5158 - val_loss: 1.5537 - val_accuracy: 0.3547
Epoch 60/100
59/59 [==============================] - 7s 119ms/step - loss: 1.1399 - accuracy: 0.5169 - val_loss: 1.6722 - val_accuracy: 0.3568
Epoch 61/100
59/59 [==============================] - 7s 118ms/step - loss: 1.1347 - accuracy: 0.5222 - val_loss: 1.6592 - val_accuracy: 0.3419
Epoch 62/100
59/59 [==============================] - 7s 123ms/step - loss: 1.1127 - accuracy: 0.5211 - val_loss: 1.6760 - val_accuracy: 0.3675
Epoch 63/100
59/59 [==============================] - 7s 117ms/step - loss: 1.1399 - accuracy: 0.5254 - val_loss: 1.7019 - val_accuracy: 0.3568
Epoch 64/100
59/59 [==============================] - 7s 118ms/step - loss: 1.1107 - accuracy: 0.5243 - val_loss: 1.7056 - val_accuracy:

In [38]:
# Extract features and labels
X_word2vec = np.array(df['word2vec_features'].tolist())

X_word2vec = X_word2vec.reshape(X_word2vec.shape[0], X_word2vec.shape[1], 1)

# Split data into train and test sets
X_train_word2vec, X_test_word2vec, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.2, random_state=42)

# Define GRU model
model_word2vec = Sequential([
    GRU(128, input_shape=(X_word2vec.shape[1], X_word2vec.shape[2]), activation='relu'),
    Dense(5, activation='softmax')
])

# Compile the model
model_word2vec.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_word2vec.fit(X_train_word2vec, y_train, epochs=10, validation_data=(X_test_word2vec, y_test))

# Evaluate the model
y_pred_word2vec = np.argmax(model_word2vec.predict(X_test_word2vec), axis=-1)
accuracy_word2vec = accuracy_score(y_test, y_pred_word2vec)
recall_word2vec = recall_score(y_test, y_pred_word2vec, average='weighted')
mcc_word2vec = matthews_corrcoef(y_test, y_pred_word2vec)
f1_word2vec = f1_score(y_test, y_pred_word2vec, average='weighted')

print(accuracy_word2vec)
print(f1_word2vec)

Epoch 1/10
59/59 [==============================] - 10s 125ms/step - loss: 1.6099 - accuracy: 0.2108 - val_loss: 1.6107 - val_accuracy: 0.1880
Epoch 2/10
59/59 [==============================] - 7s 117ms/step - loss: 1.6093 - accuracy: 0.2108 - val_loss: 1.6106 - val_accuracy: 0.1880
Epoch 3/10
59/59 [==============================] - 7s 116ms/step - loss: 1.6094 - accuracy: 0.2108 - val_loss: 1.6101 - val_accuracy: 0.1880
Epoch 4/10
59/59 [==============================] - 7s 118ms/step - loss: 1.6094 - accuracy: 0.2108 - val_loss: 1.6103 - val_accuracy: 0.1880
Epoch 5/10
59/59 [==============================] - 7s 115ms/step - loss: 1.6091 - accuracy: 0.2108 - val_loss: 1.6105 - val_accuracy: 0.1880
Epoch 6/10
59/59 [==============================] - 7s 117ms/step - loss: 1.6091 - accuracy: 0.2108 - val_loss: 1.6104 - val_accuracy: 0.1880
Epoch 7/10
59/59 [==============================] - 7s 115ms/step - loss: 1.6093 - accuracy: 0.2108 - val_loss: 1.6101 - val_accuracy: 0.1880
Epoch

In [39]:
# Assuming you have a DataFrame named df

# Extract features and labels
X_fasttext = np.array(df['fasttext_embeddings'].tolist())

X_fasttext = X_fasttext.reshape(X_fasttext.shape[0], X_fasttext.shape[1], 1)
# Split data into train and test sets
X_train_fasttext, X_test_fasttext, y_train, y_test = train_test_split(X_fasttext, y, test_size=0.2, random_state=42)

# Define GRU model
model_fasttext = Sequential([
    GRU(128, input_shape=(X_fasttext.shape[1], X_fasttext.shape[2]), activation='relu'),
    Dense(5, activation='softmax')
])

# Compile the model
model_fasttext.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_fasttext.fit(X_train_fasttext, y_train, epochs=10, validation_data=(X_test_fasttext, y_test))

# Evaluate the model
y_pred_fasttext = np.argmax(model_fasttext.predict(X_test_fasttext), axis=-1)
accuracy_fasttext = accuracy_score(y_test, y_pred_fasttext)
recall_fasttext = recall_score(y_test, y_pred_fasttext, average='weighted')
mcc_fasttext = matthews_corrcoef(y_test, y_pred_fasttext)
f1_fasttext = f1_score(y_test, y_pred_fasttext, average='weighted')

Epoch 1/10
59/59 [==============================] - 10s 123ms/step - loss: 1.6096 - accuracy: 0.2065 - val_loss: 1.6081 - val_accuracy: 0.1880
Epoch 2/10
59/59 [==============================] - 7s 120ms/step - loss: 1.6059 - accuracy: 0.2081 - val_loss: 1.6041 - val_accuracy: 0.2073
Epoch 3/10
59/59 [==============================] - 7s 119ms/step - loss: 1.6005 - accuracy: 0.2253 - val_loss: 1.5992 - val_accuracy: 0.2244
Epoch 4/10
59/59 [==============================] - 7s 119ms/step - loss: 1.5933 - accuracy: 0.2285 - val_loss: 1.5898 - val_accuracy: 0.2500
Epoch 5/10
59/59 [==============================] - 7s 117ms/step - loss: 1.5870 - accuracy: 0.2317 - val_loss: 1.5882 - val_accuracy: 0.2393
Epoch 6/10
59/59 [==============================] - 7s 118ms/step - loss: 1.5894 - accuracy: 0.2365 - val_loss: 1.5969 - val_accuracy: 0.2158
Epoch 7/10
59/59 [==============================] - 7s 116ms/step - loss: 1.5841 - accuracy: 0.2327 - val_loss: 1.5904 - val_accuracy: 0.2628
Epoch

In [82]:
# Reshape the data to have a third dimension
from tensorflow.keras.layers import Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D
X_tfidf = X_tfidf.reshape(X_tfidf.shape[0], X_tfidf.shape[1], 1)

# Concatenate all feature types
X_combined = np.hstack((X_tfidf, X_glove, X_word2vec, X_fasttext))

# Split data into train and test sets
X_train_combined, X_test_combined, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Define GRU model
model_combined = Sequential([
    GRU(128, input_shape=(X_combined.shape[1],X_combined.shape[2]), activation='relu'),
    Dense(5, activation='softmax')
])

# #Optimum model
# model_combined = Sequential([
#     Bidirectional(GRU(128, return_sequences=True), input_shape=(X_combined.shape[1], X_combined.shape[2])),
#     Conv1D(64, 3, activation='relu'),  # Convolutional layer for text data
#     GlobalMaxPooling1D(),  # Pooling layer
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(5, activation='softmax')
# ])


# Compile the model
model_combined.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_combined.fit(X_train_combined, y_train, epochs=10, validation_data=(X_test_combined, y_test))

# Evaluate the model

y_pred_combined = np.argmax(model_combined.predict(X_test_combined), axis=-1)
accuracy_combined = accuracy_score(y_test, y_pred_combined)
recall_combined = recall_score(y_test, y_pred_combined, average='weighted')
mcc_combined = matthews_corrcoef(y_test, y_pred_combined)
f1_combined = f1_score(y_test, y_pred_combined, average='weighted')


Epoch 1/10
59/59 [==============================] - 32s 483ms/step - loss: 1.6163 - accuracy: 0.2001 - val_loss: 1.6035 - val_accuracy: 0.2201
Epoch 2/10
59/59 [==============================] - 27s 462ms/step - loss: 1.6034 - accuracy: 0.2172 - val_loss: 1.6005 - val_accuracy: 0.2201
Epoch 3/10
14/59 [======>.......................] - ETA: 19s - loss: 1.6019 - accuracy: 0.2254

KeyboardInterrupt: 

In [78]:
df.head()

,text,sentiment,tfidf_features,glove_features,word2vec_features,fasttext_embeddings
0,covid change work general recruiting specifica...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.13231022138562468, -0.02969788842731052, -...","[-0.01437218114733696, 0.014927376993000507, 0...","[0.09200902283191681, 1.7944399118423462, -4.8..."
1,wear face covering shopping includes visit loc...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.1756293624639511, 0.13796843588352203, -0....","[-0.006316069047898054, 0.010635077022016048, ...","[0.0926610603928566, 1.7999165058135986, -4.94..."
2,order logo graphicdesigner logodesign logodesi...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.007864626124501228, 0.26375137781724334, -...","[0.0018355390056967735, 0.0016172656323760748,...","[0.055948611348867416, 1.1404274702072144, -3...."
3,rajasthan government today started plasma bank...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.15869990789464541, -0.146028643740075, 0.1...","[-0.012957141734659672, 0.01843681000173092, 0...","[0.18618302047252655, 2.992691993713379, -8.20..."
4,nagaland police covid awareness city tower jun...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.03638550266623497, -0.22403829880058765, 0....","[-0.015640147030353546, 0.023020263761281967, ...","[0.08334425091743469, 1.768351674079895, -4.75..."


In [79]:
import pandas as pd

# Load the CSV file into a DataFrame
#df = pd.read_csv("C:\\Users\\lenovo\\My_Folders\\7th_sem\\IT350\\IT350_Project\\Mainprocessed_covid19_tweets_copy 2.csv")

# Select the 10th and 14th columns
selected_columns = df.iloc[:, [0, 1]]

# Create a new DataFrame with the selected columns
df = pd.DataFrame(selected_columns)

# Display the new DataFrame
df.head()  # You can adjust the number of rows to display if needed

,text,sentiment
0,covid change work general recruiting specifica...,1
1,wear face covering shopping includes visit loc...,1
2,order logo graphicdesigner logodesign logodesi...,1
3,rajasthan government today started plasma bank...,1
4,nagaland police covid awareness city tower jun...,1


In [81]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Concatenate
from tensorflow.keras.utils import to_categorical

# Assuming you have a DataFrame named 'df' with columns 'text' and 'sentiment_label'
# And you have tfidf_features, glove_features, and word2vec_features extracted



# Combine the extracted features
combined_features = np.concatenate((tfidf_features, glove_features, word2vec_features), axis=1)

# Define your labels and features
labels = df['sentiment'].values

# Convert labels to one-hot encoding
labels_encoded = to_categorical(labels)

# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(combined_features, labels_encoded, test_size=0.2, random_state=42)

# # Define the ensemble model
# ensemble_model = Sequential([
#     Dense(128, activation='relu', input_shape=(300,)),
#     Dense(64, activation='relu'),
#     Dense(5, activation='softmax')
# ])
ensemble_model = Sequential([
    Dense(128, activation='tanh', input_shape=(300,)),
    Dense(64, activation='tanh'),
    Dense(5, activation='sigmoid')
])

# Compile the model
ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
ensemble_model.fit(X_train, y_train, epochs=100, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = ensemble_model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy}')

# Make predictions
predictions = ensemble_model.predict(X_test)

# Convert predictions back to labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_labels)
print(f'Accuracy: {accuracy}')

Epoch 1/100
47/47 [==============================] - 2s 13ms/step - loss: 1.5006 - accuracy: 0.3438 - val_loss: 1.3821 - val_accuracy: 0.4225
Epoch 2/100
47/47 [==============================] - 0s 6ms/step - loss: 1.2723 - accuracy: 0.4736 - val_loss: 1.2804 - val_accuracy: 0.4893
Epoch 3/100
47/47 [==============================] - 0s 6ms/step - loss: 1.1653 - accuracy: 0.5164 - val_loss: 1.2768 - val_accuracy: 0.4545
Epoch 4/100
47/47 [==============================] - 0s 6ms/step - loss: 1.0918 - accuracy: 0.5659 - val_loss: 1.2520 - val_accuracy: 0.4759
Epoch 5/100
47/47 [==============================] - 0s 6ms/step - loss: 1.0409 - accuracy: 0.5926 - val_loss: 1.2720 - val_accuracy: 0.4920
Epoch 6/100
47/47 [==============================] - 0s 6ms/step - loss: 1.0097 - accuracy: 0.6033 - val_loss: 1.3112 - val_accuracy: 0.4439
Epoch 7/100
47/47 [==============================] - 0s 6ms/step - loss: 0.9958 - accuracy: 0.6087 - val_loss: 1.2815 - val_accuracy: 0.4973
Epoch 8/100


Epoch 59/100
47/47 [==============================] - 0s 6ms/step - loss: 0.6273 - accuracy: 0.7639 - val_loss: 1.7466 - val_accuracy: 0.4519
Epoch 60/100
47/47 [==============================] - 0s 6ms/step - loss: 0.6187 - accuracy: 0.7659 - val_loss: 1.8195 - val_accuracy: 0.4572
Epoch 61/100
47/47 [==============================] - 0s 6ms/step - loss: 0.6243 - accuracy: 0.7659 - val_loss: 1.8048 - val_accuracy: 0.4599
Epoch 62/100
47/47 [==============================] - 0s 6ms/step - loss: 0.5995 - accuracy: 0.7719 - val_loss: 1.7866 - val_accuracy: 0.4706
Epoch 63/100
47/47 [==============================] - 0s 6ms/step - loss: 0.5792 - accuracy: 0.7853 - val_loss: 1.8032 - val_accuracy: 0.4599
Epoch 64/100
47/47 [==============================] - 0s 6ms/step - loss: 0.5673 - accuracy: 0.7853 - val_loss: 1.8703 - val_accuracy: 0.4412
Epoch 65/100
47/47 [==============================] - 0s 6ms/step - loss: 0.5672 - accuracy: 0.7946 - val_loss: 1.8507 - val_accuracy: 0.4652
Epoch 

In [72]:
unique_values = df['sentiment'].unique()
print(unique_values)


[nan]


In [73]:
df['sentiment']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2332   NaN
2333   NaN
2334   NaN
2335   NaN
2336   NaN
Name: sentiment, Length: 2337, dtype: float64